In [112]:
from IPython.utils import io
import csv
from sklearn.model_selection import cross_val_predict, cross_val_score
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import SMOTE
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/TF-IDF/TF-IDF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
   #%run ../../2-FeatureExtraction/ContextualEmbeddings/ContextualEmbeddings.ipynb import x, X, y1, y2, Y1, Y2, Features, Preprocesss
if Features["method"] ==  "contextual":
   x, X = x[:,0,:], X[:,0,:]


In [113]:
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

#### SVM Training

In [114]:
kernel = 'rbf'
WeightedLoss = True
Oversampling = True
CrossValidation = False

model = svm.SVC(kernel=kernel, class_weight='balanced' if WeightedLoss else None )

ModelInfo = {
   "Model": "SVM-1",
   "Kernel":kernel,
   "Weighted Loss": WeightedLoss,
   "Oversampling": Oversampling,
   "CrossValidation": CrossValidation,
}

if Oversampling:
   sm = SMOTE(random_state=42)
   x, y1 = sm.fit_resample(x, y1)

#### SVM Prediction

In [115]:
if CrossValidation:
   X_c, Y_c1 = np.concatenate([x, X]), np.concatenate([y1, Y1])
   outputs = cross_val_predict(model, X_c, Y_c1, cv=5)
   Report = classification_report(Y_c1, outputs, output_dict=True)
   print(classification_report(Y_c1, outputs))
   F1 = f1_score(Y_c1, outputs, average='macro')
   print('F1 score: ', F1)

else:
   model = model.fit(x, y1)
   outputs = model.predict(X)
   Report = classification_report(Y1, outputs, output_dict=True)
   print(classification_report(Y1, outputs))
   F1 = f1_score(Y1, outputs, average='macro')
   print('F1 score: ', F1)

              precision    recall  f1-score   support

           0       0.34      0.38      0.36       126
           1       0.87      0.89      0.88       804
           2       0.47      0.27      0.35        70

    accuracy                           0.78      1000
   macro avg       0.56      0.51      0.53      1000
weighted avg       0.78      0.78      0.78      1000

F1 score:  0.5286965062785952


#### Log Model Info

In [116]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)